In [1]:
import pandas as pd
import numpy as np 

In [3]:
df=pd.read_csv("Breast_Cancer.csv")

In [4]:
df.head()

,Age,Race,Marital Status,T Stage,N Stage,6th Stage,differentiate,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Survival Months,Status
0,68,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,4,Positive,Positive,24,1,60,Alive
1,50,White,Married,T2,N2,IIIA,Moderately differentiated,2,Regional,35,Positive,Positive,14,5,62,Alive
2,58,White,Divorced,T3,N3,IIIC,Moderately differentiated,2,Regional,63,Positive,Positive,14,7,75,Alive
3,58,White,Married,T1,N1,IIA,Poorly differentiated,3,Regional,18,Positive,Positive,2,1,84,Alive
4,47,White,Married,T2,N1,IIB,Poorly differentiated,3,Regional,41,Positive,Positive,3,1,50,Alive


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4024 entries, 0 to 4023
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Age                     4024 non-null   int64 
 1   Race                    4024 non-null   object
 2   Marital Status          4024 non-null   object
 3   T Stage                 4024 non-null   object
 4   N Stage                 4024 non-null   object
 5   6th Stage               4024 non-null   object
 6   differentiate           4024 non-null   object
 7   Grade                   4024 non-null   object
 8   A Stage                 4024 non-null   object
 9   Tumor Size              4024 non-null   int64 
 10  Estrogen Status         4024 non-null   object
 11  Progesterone Status     4024 non-null   object
 12  Regional Node Examined  4024 non-null   int64 
 13  Reginol Node Positive   4024 non-null   int64 
 14  Survival Months         4024 non-null   int64 
 15  Stat

In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Age,4024.0,53.972167,8.963134,30.0,47.0,54.0,61.0,69.0
Tumor Size,4024.0,30.473658,21.119696,1.0,16.0,25.0,38.0,140.0
Regional Node Examined,4024.0,14.357107,8.099675,1.0,9.0,14.0,19.0,61.0
Reginol Node Positive,4024.0,4.158052,5.109331,1.0,1.0,2.0,5.0,46.0
Survival Months,4024.0,71.297962,22.921430,1.0,56.0,73.0,90.0,107.0


In [11]:
object_data=df.select_dtypes(include='object').columns
int_data=df.select_dtypes(include='int64').columns

In [13]:
def TNM_Tumor_Stage(dataset , var1 , var2 , var3):
    TNM =dataset[var1] +"-"+ dataset[var2] +"-"+ dataset[var3]
    return TNM
df["TNM Tumor Stage"]=TNM_Tumor_Stage(df , 'T Stage ', 'N Stage', 'A Stage')

In [15]:
def Hormon_State(dataset , var1 , var2 ):
    return  ["ER-"] +  dataset[var1]+ " "+["PR-"]+dataset[var2]
df["Hormon Status"]=Hormon_State(df , 'Estrogen Status', 'Progesterone Status')

In [16]:
def Lymph_node_Ratio(dataset , var1  , var2):
    return round(dataset[var1] / dataset[var2] , 2)
df["Lymph_Node_Ratio"]=Lymph_node_Ratio(df , 'Reginol Node Positive', 'Regional Node Examined')

In [17]:
def tumor_size(tumor_size):
    if tumor_size <= 20:
        return "Small"
    elif 20< tumor_size <= 50:
        return "Medium"
    elif tumor_size > 50: 
        return "Large"
df['Tumor Size Category'] = df['Tumor Size'].apply(tumor_size)

In [18]:
def month_to_years(var):
        year_survived=var/12
        return round(year_survived , 1)
df['Year_Survived'] = df['Survival Months'].apply(month_to_years)

In [19]:
df['Status'] = df['Status'].map({"Alive":1 , "Dead":0})

In [21]:
from sklearn.model_selection import train_test_split
y=df["Status"]
X=df.drop('Status' , axis=1)
X_train , X_test , y_train , y_test=train_test_split(X , y , test_size=0.2 , random_state=42, stratify = y)

In [23]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler , OrdinalEncoder , LabelEncoder
from sklearn.compose import ColumnTransformer
list_int=list(X_train[['Age', 'Tumor Size', 'Regional Node Examined', 'Reginol Node Positive','Survival Months','Lymph_Node_Ratio' , 'Year_Survived']])
list_1hot=list(X_train[['Race', 'Marital Status', 'differentiate', 'Estrogen Status', 'Progesterone Status' , 'TNM Tumor Stage' , 'Hormon Status', ]])
list_ordinal=list(X_train[[ 'T Stage ', 'N Stage', '6th Stage', 'Grade', 'A Stage', 'Tumor Size Category']])


coltransformer= ColumnTransformer([ ('std'  ,MinMaxScaler() , list_int) , 
                                    ('1hot' ,OneHotEncoder() , list_1hot) , 
                                     ('ordinal', OrdinalEncoder(), list_ordinal), 
                                  ])

In [24]:
X_train_pre=coltransformer.fit_transform(X_train)

In [27]:
pip install imblearn

   ---------------------------------------- 0.0/258.3 kB ? eta -:--:--
   - -------------------------------------- 10.2/258.3 kB ? eta -:--:--
   ------ -------------------------------- 41.0/258.3 kB 653.6 kB/s eta 0:00:01
   ---------------------------------------  256.0/258.3 kB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 258.3/258.3 kB 2.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: C:\Users\pravi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [28]:
#the SMOTE for oversampling imbalanced classification datasets.

from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_pre, y_train)

C:\Users\pravi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\pravi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [30]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=600 , max_depth=17)

In [33]:
#RandomForest
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_score , recall_score , confusion_matrix
y_pred=cross_val_predict(rf , X_train_resampled, y_train_resampled , cv=5)
print(f'the recall score is : {recall_score(y_train_resampled , y_pred)}')
print(f'the precision score is : {precision_score(y_train_resampled , y_pred)}')

the recall score is : 0.9325018341892883
the precision score is : 0.9267225665329931


In [34]:
import pickle

with open("breast_cancer.pkl", 'wb') as file:
    pickle.dump(rf, file)